In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
class MnistModel(): #tf.Module
  def __init__(self):

    self.loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.opt = tf.optimizers.Adam()

  def build(self):

    self.model = Sequential()

    self.model.add(Conv2D(28, (3, 3), input_shape=(28,28,1), padding='same')) #train_imgs.shape[1:]
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(pool_size=(2, 2)))

    self.model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    self.model.add(Dense(64))

    self.model.add(Dense(9))
    #self.model.add(Activation('sigmoid'))

    return self.model

  def apply_softmax(self, logits):
    return tf.nn.softmax(logits)

  def train(self, train_data):

    for step, (img_batch, lbl_batch) in enumerate(train_data):

      with tf.GradientTape() as tape:
          logits = self.model(img_batch)
          softmx = self.apply_softmax(logits)
          xent = self.loss_fn(lbl_batch, softmx)

      varis = self.model.trainable_variables
      grads = tape.gradient(xent, varis)

      self.opt.apply_gradients(zip(grads, varis))
      
      self.train_acc_metric(lbl_batch, logits)
      
      if tf.equal(step % 500, 0): #not step % 500:
        acc = self.train_acc_metric.result()
        tf.print("Loss: {} Accuracy: {}".format(xent, acc))
        self.train_acc_metric.reset_states()

  def test(self,test_data, temp=None):
    for img_batch, lbl_batch in test_data:
      logits = self.model(img_batch)
      if not temp is None:
        logits = tf.math.divide(logits, temp)
      pred = self.apply_softmax(logits)
      self.test_acc_metric(lbl_batch, pred)
    tf.print("Test acc: {}".format(self.test_acc_metric.result()))

  def save_model(self, path):
    self.model.save(path)

  def load_model(self, path):
    self.model = tf.keras.models.load_model(path)
    return self.model

  def get_max_pred_value(self, data, temp=None):
    logits = self.model(data)
    if not temp is None:
      logits = tf.math.divide(logits, temp)
    pred = self.apply_softmax(logits)
    return np.max(pred)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
os.chdir("/content/drive/My Drive/Uni Projects/Anomaly Detection using Ensembles/Experiments/")
print(os.getcwd())

/content
/content/drive/My Drive/Uni Projects/Anomaly Detection using Ensembles/Experiments


# Model Training

In [ ]:
(train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.mnist.load_data()
train_imgs = (train_imgs.astype(np.float32) / 255.).reshape((-1, 28, 28, 1))
train_lbls = train_lbls.astype(np.int32)


test_imgs = (test_imgs.astype(np.float32) / 255.).reshape((-1, 28, 28, 1))
test_lbls = test_lbls.astype(np.int32)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
train_images, val_images, train_labels, val_labels = train_test_split(train_imgs, train_lbls, test_size=0.20)

In [ ]:
import pickle

In [ ]:
new_train_images = []
new_train_labels = []
for ind, im in enumerate(train_images):
  if train_labels[ind] != 9:
    new_train_images.append(im)
    new_train_labels.append(train_labels[ind])

new_train_images = np.asarray(new_train_images)
new_train_labels = np.asarray(new_train_labels)

In [ ]:
new_val_images = []
new_val_labels = []
for ind, im in enumerate(val_images):
  if val_labels[ind] != 9:
    new_val_images.append(im)
    new_val_labels.append(val_labels[ind])

new_val_images = np.asarray(new_val_images)
new_val_labels = np.asarray(new_val_labels)

In [ ]:
pickle_out = open("mnist_data_wo9.pickle","wb")
pickle.dump([new_train_images, new_train_labels, new_val_images, new_val_labels, test_imgs, test_lbls], pickle_out)
pickle_out.close()


In [ ]:
pickle_in = open("mnist_data_wo9.pickle","rb")
x_train, y_train, x_val, y_val, x_test, y_test = pickle.load(pickle_in)

Method to separate leave out class

In [ ]:
def get_train_data(anom_class):
  class_train_images = []
  class_train_labels = []
  for ind, im in enumerate(new_train_images):
    if new_train_labels[ind] != anom_class:
      class_train_images.append(im)
      class_train_labels.append(new_train_labels[ind])


  class_train_images = np.asarray(class_train_images)
  class_train_labels = np.asarray(class_train_labels)

  print(set(class_train_labels))
  print(class_train_labels.shape)

  train_data = tf.data.Dataset.from_tensor_slices((class_train_images, class_train_labels))

  train_data = train_data.shuffle(class_train_images.shape[0])
  train_data = train_data.batch(128)
  train_data = train_data.repeat(10)

  return train_data
  #{
   #   'class_train_images': class_train_images,
   #   'class_train_labels': class_train_labels,
  #}

In [ ]:
train_data0 = get_train_data(0)
cnnmodel0 = MnistModel()
class_0 = cnnmodel0.build()
cnnmodel0.train(train_data0)

train_data1 = get_train_data(1)
cnnmodel1 = MnistModel()
class_1 = cnnmodel1.build()
cnnmodel1.train(train_data1)


train_data2 = get_train_data(2)
cnnmodel2 = MnistModel()
class_2 = cnnmodel2.build()
cnnmodel2.train(train_data2)


train_data3 = get_train_data(3)
cnnmodel3 = MnistModel()
class_3 = cnnmodel3.build()
cnnmodel3.train(train_data3)


train_data4 = get_train_data(4)
cnnmodel4 = MnistModel()
class_4 = cnnmodel4.build()
cnnmodel4.train(train_data4)


train_data5 = get_train_data(5)
cnnmodel5 = MnistModel()
class_5 = cnnmodel5.build()
cnnmodel5.train(train_data5)


train_data6 = get_train_data(6)
cnnmodel6 = MnistModel()
class_6 = cnnmodel6.build()
cnnmodel6.train(train_data6)


train_data7 = get_train_data(7)
cnnmodel7 = MnistModel()
class_7 = cnnmodel7.build()
cnnmodel7.train(train_data7)


train_data8 = get_train_data(8)
cnnmodel8 = MnistModel()
class_8 = cnnmodel8.build()
cnnmodel8.train(train_data8)

{1, 2, 3, 4, 5, 6, 7, 8}
(38541,)
Loss: 2.1973538398742676 Accuracy: 0.1015625
Loss: 1.4060086011886597 Accuracy: 0.9257572293281555
Loss: 1.398244023323059 Accuracy: 0.9759134650230408
Loss: 1.3872153759002686 Accuracy: 0.9830946326255798
Loss: 1.4034329652786255 Accuracy: 0.9865767359733582
Loss: 1.387589693069458 Accuracy: 0.9896032810211182
Loss: 1.3769629001617432 Accuracy: 0.990936815738678
{0, 2, 3, 4, 5, 6, 7, 8}
(37816,)
Loss: 2.1985247135162354 Accuracy: 0.15625
Loss: 1.4076400995254517 Accuracy: 0.9313446283340454
Loss: 1.3868271112442017 Accuracy: 0.9770107865333557
Loss: 1.398125410079956 Accuracy: 0.9841675162315369
Loss: 1.3914425373077393 Accuracy: 0.9873920679092407
Loss: 1.373413324356079 Accuracy: 0.9901653528213501
{0, 1, 3, 4, 5, 6, 7, 8}
(38487,)
Loss: 2.1960530281066895 Accuracy: 0.1171875
Loss: 1.4157180786132812 Accuracy: 0.9350364804267883
Loss: 1.3988921642303467 Accuracy: 0.9785662889480591
Loss: 1.3761837482452393 Accuracy: 0.984787106513977
Loss: 1.3784637

In [ ]:
class_0.save_weights("classifier0")
class_1.save_weights("classifier1")
class_2.save_weights("classifier2")
class_3.save_weights("classifier3")
class_4.save_weights("classifier4")
class_5.save_weights("classifier5")
class_6.save_weights("classifier6")
class_7.save_weights("classifier7")
class_8.save_weights("classifier8")

In [ ]:
class_0.save("classifier0")
class_1.save("classifier1")
class_2.save("classifier2")
class_3.save("classifier3")
class_4.save("classifier4")
class_5.save("classifier5")
class_6.save("classifier6")
class_7.save("classifier7")
class_8.save("classifier8")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: classifier0/assets
INFO:tensorflow:Assets written to: classifier1/assets
INFO:tensorflow:Assets written to: classifier2/assets
INFO:tensorflow:Assets written to: classifier3/assets
INFO:tensorflow:Assets written to: classifier4/assets
INFO:tensorflow:Assets written to: classifier5/assets
INFO:tensorflow:Assets written to: classifier6/assets
INFO:tensorflow:Assets written to: classifier7/assets
INFO:tensorflow:Assets written to: classifier8/assets


In [ ]:
test_model = tf.keras.models.load_model("classifier0")

## Temperature Scaling

In [ ]:
def temp_cal(y_pred):

  temp = tf.Variable(initial_value=1.0, trainable=True, dtype=tf.float32) 

  def compute_loss():
      y_pred_model_w_temp = tf.math.divide(y_pred, temp)
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                  tf.convert_to_tensor(keras.utils.to_categorical(new_val_labels)), y_pred_model_w_temp))
      return loss

  optimizer = tf.optimizers.Adam(learning_rate=0.01)

  print('Temperature Initial value: {}'.format(temp.numpy()))

  for i in range(300):
      opts = optimizer.minimize(compute_loss, var_list=[temp])


  print('Temperature Final value: {}'.format(temp.numpy()))

  return temp

In [ ]:
def temp_scaling(y_pred, temp):
  return tf.math.divide(y_pred, temp)

In [ ]:
y_pred0 = class_0.predict(new_val_images)
y_pred1 = class_1.predict(new_val_images)
y_pred2 = class_2.predict(new_val_images)
y_pred3 = class_3.predict(new_val_images)
y_pred4 = class_4.predict(new_val_images)
y_pred5 = class_5.predict(new_val_images)
y_pred6 = class_6.predict(new_val_images)
y_pred7 = class_7.predict(new_val_images)
y_pred8 = class_8.predict(new_val_images)

In [ ]:
from tensorflow import keras

In [ ]:
temp0 = temp_cal(y_pred0)
temp1 = temp_cal(y_pred1)
temp2 = temp_cal(y_pred2)
temp3 = temp_cal(y_pred3)
temp4 = temp_cal(y_pred4)
temp5 = temp_cal(y_pred5)
temp6 = temp_cal(y_pred6)
temp7 = temp_cal(y_pred7)
temp8 = temp_cal(y_pred8)

Temperature Initial value: 1.0
Temperature Final value: 2.6913704872131348
Temperature Initial value: 1.0
Temperature Final value: 2.647186040878296
Temperature Initial value: 1.0
Temperature Final value: 2.69714093208313
Temperature Initial value: 1.0
Temperature Final value: 2.703646659851074
Temperature Initial value: 1.0
Temperature Final value: 2.6812326908111572
Temperature Initial value: 1.0
Temperature Final value: 2.7035343647003174
Temperature Initial value: 1.0
Temperature Final value: 2.681792736053467
Temperature Initial value: 1.0
Temperature Final value: 2.6865651607513428
Temperature Initial value: 1.0
Temperature Final value: 2.6931824684143066


In [ ]:
pickle_out = open("temperature_value.pickle","wb")
pickle.dump([temp0, temp1, temp2, temp3, temp4, temp5, temp6, temp7, temp8], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("temperature_value.pickle","rb")
temp_test = pickle.load(pickle_in)

In [ ]:
temp_test

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6913705>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.647186>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.697141>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.7036467>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6812327>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.7035344>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6817927>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6865652>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6931825>]

# Calculating predefined stuff

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
os.chdir("/content/drive/My Drive/Uni Projects/Anomaly Detection using Ensembles/Experiments/")
print(os.getcwd())

/content/drive/My Drive/Uni Projects/Anomaly Detection using Ensembles/Experiments
/content/drive/My Drive/Uni Projects/Anomaly Detection using Ensembles/Experiments


In [ ]:
from models import MnistModel

In [ ]:
model0 = MnistModel()
classifier0 = model0.build()
classifier0 = model0.load_model("classifier0")

model1 = MnistModel()
classifier1 = model1.build()
classifier1 = model1.load_model("classifier1")

model2 = MnistModel()
classifier2 = model2.build()
classifier2 = model2.load_model("classifier2")

model3 = MnistModel()
classifier3 = model3.build()
classifier3 = model3.load_model("classifier3")

model4 = MnistModel()
classifier4 = model4.build()
classifier4 = model4.load_model("classifier4")

model5 = MnistModel()
classifier5 = model5.build()
classifier5 = model5.load_model("classifier5")

model6 = MnistModel()
classifier6 = model6.build()
classifier6 = model6.load_model("classifier6")

model7 = MnistModel()
classifier7 = model7.build()
classifier7 = model7.load_model("classifier7")

model8 = MnistModel()
classifier8 = model8.build()
classifier8 = model8.load_model("classifier8")

In [ ]:
import pickle
pickle_in = open("mnist_data_wo9.pickle","rb")
x_train, y_train, x_val, y_val, x_test, y_test = pickle.load(pickle_in)

In [ ]:
pickle_in = open("temperature_value.pickle","rb")
temp_value = pickle.load(pickle_in)

In [ ]:
temp_value

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6913705>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.647186>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.697141>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.7036467>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6812327>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.7035344>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6817927>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6865652>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.6931825>]

In [ ]:
x_test[0].shape

(28, 28, 1)

In [ ]:
data = x_test[1].reshape([-1, 28, 28, 1])
logits = model0.model(data)
#if not temp is None:
#  logits = tf.math.divide(logits, temp)
pred = model0.apply_softmax(logits)
print(np.max(pred), np.argmax(pred), y_test[1])

1.0 2 2


## Reference vector for IN-distribution

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(x_val,y_val)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model0.model(img)
    pred = model0.apply_softmax(logits)
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model1.model(img)
    pred = model1.apply_softmax(logits)
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 2:
    logits = model2.model(img)
    pred = model2.apply_softmax(logits)
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 3:
    logits = model3.model(img)
    pred = model3.apply_softmax(logits)
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 4:
    logits = model4.model(img)
    pred = model4.apply_softmax(logits)
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 5:
    logits = model5.model(img)
    pred = model5.apply_softmax(logits)
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 6:
    logits = model6.model(img)
    pred = model6.apply_softmax(logits)
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 7:
    logits = model7.model(img)
    pred = model7.apply_softmax(logits)
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 8:
    logits = model8.model(img)
    pred = model8.apply_softmax(logits)
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)

classifier_avg = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg[i]

treshold = treshold_value/len(classifier_avg)

print('Threshold:', treshold)

10817it [02:12, 81.69it/s]

[9530.353578180075, 9439.172650545835, 9558.319529503584, 9537.780795872211, 9609.451030313969, 9679.753895014524, 9607.5714263618, 9509.476637899876, 9604.689707696438]
[9587, 9493, 9606, 9588, 9678, 9733, 9660, 9552, 9639]
Each Classifier Average:  [0.9940913297361088, 0.9943297851623129, 0.9950363865816765, 0.994762285760556, 0.9929170314438901, 0.994529322409794, 0.994572611424617, 0.9955482242357492, 0.9964404718016846]
Threshold: 0.9946919387284877


In [ ]:
pickle_out = open("reference_vector.pickle","wb")
pickle.dump(classifier_avg, pickle_out)
pickle_out.close()

pickle_out = open("threshold_value.pickle","wb")
pickle.dump(treshold, pickle_out)
pickle_out.close()

In [ ]:
logits = tf.math.divide(logits, temp)

## Reference vector for IN-distribution (With temperature Scaling)

In [ ]:
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(x_val,y_val)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model0.model(img)
    logits = tf.math.divide(logits, temp_value[0])
    pred = model0.apply_softmax(logits)
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model1.model(img)
    logits = tf.math.divide(logits, temp_value[1])
    pred = model1.apply_softmax(logits)
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 2:
    logits = model2.model(img)
    logits = tf.math.divide(logits, temp_value[2])
    pred = model2.apply_softmax(logits)
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 3:
    logits = model3.model(img)
    logits = tf.math.divide(logits, temp_value[3])
    pred = model3.apply_softmax(logits)
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 4:
    logits = model4.model(img)
    logits = tf.math.divide(logits, temp_value[4])
    pred = model4.apply_softmax(logits)
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 5:
    logits = model5.model(img)
    logits = tf.math.divide(logits, temp_value[5])
    pred = model5.apply_softmax(logits)
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 6:
    logits = model6.model(img)
    logits = tf.math.divide(logits, temp_value[6])
    pred = model6.apply_softmax(logits)
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 7:
    logits = model7.model(img)
    logits = tf.math.divide(logits, temp_value[7])
    pred = model7.apply_softmax(logits)
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 8:
    logits = model8.model(img)
    logits = tf.math.divide(logits, temp_value[8])
    pred = model8.apply_softmax(logits)
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

threshold_t = treshold_value/len(classifier_avg_t)

print('Threshold(with temp scaling):', threshold_t)

10817it [02:23, 75.31it/s]

[9327.888777315617, 9267.6044062078, 9384.744804888964, 9361.287457317114, 9390.387957334518, 9492.689458370209, 9430.795419931412, 9323.725630134344, 9463.345777064562]
[9587, 9493, 9606, 9588, 9678, 9733, 9660, 9552, 9639]
Each Classifier Average:  [0.9729726480980094, 0.9762566529240282, 0.9769669794804251, 0.9763545533288605, 0.9702818720122461, 0.9753097152337623, 0.9762728177982828, 0.9761019294529255, 0.9817767171972779]
Threshold(with temp scaling): 0.9758104317250909


In [ ]:
pickle_out = open("reference_vector.pickle","wb")
pickle.dump([classifier_avg, classifier_avg_t], pickle_out)
pickle_out.close()

pickle_out = open("threshold_value.pickle","wb")
pickle.dump([treshold, threshold_t], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("threshold_value.pickle","rb")
thres_value = pickle.load(pickle_in)

In [ ]:
thres_value

[0.9946919387284877, 0.9758104317250909]

## Reference vector for Out-distribution

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(x_val,y_val)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model0.model(img)
    pred = model0.apply_softmax(logits)
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model1.model(img)
    pred = model1.apply_softmax(logits)
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 2:
    logits = model2.model(img)
    pred = model2.apply_softmax(logits)
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 3:
    logits = model3.model(img)
    pred = model3.apply_softmax(logits)
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 4:
    logits = model4.model(img)
    pred = model4.apply_softmax(logits)
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 5:
    logits = model5.model(img)
    pred = model5.apply_softmax(logits)
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 6:
    logits = model6.model(img)
    pred = model6.apply_softmax(logits)
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 7:
    logits = model7.model(img)
    pred = model7.apply_softmax(logits)
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 8:
    logits = model8.model(img)
    pred = model8.apply_softmax(logits)
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)

classifier_avg = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg[i]

treshold = treshold_value/len(classifier_avg)

print('Threshold:', treshold)

10817it [00:16, 658.23it/s]

[1107.3190031647682, 1123.2836446464062, 1094.5776670277119, 1122.0071937441826, 1015.7577748596668, 990.4176844656467, 1015.697742909193, 1133.181160658598, 1046.86403465271]
[1230, 1324, 1211, 1229, 1139, 1084, 1157, 1265, 1178]
Each Classifier Average:  [0.9002593521664782, 0.8484015442948687, 0.9038626482474912, 0.9129432007682527, 0.8917978708162132, 0.9136694506140652, 0.8778718607685333, 0.8957953839198403, 0.8886791465642699]
Threshold: 0.892586717573335


## Reference vector for out-distribution(With temperature scaling)

In [ ]:
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(x_val,y_val)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model0.model(img)
    logits = tf.math.divide(logits, temp_value[0])
    pred = model0.apply_softmax(logits)
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model1.model(img)
    logits = tf.math.divide(logits, temp_value[1])
    pred = model1.apply_softmax(logits)
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 2:
    logits = model2.model(img)
    logits = tf.math.divide(logits, temp_value[2])
    pred = model2.apply_softmax(logits)
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 3:
    logits = model3.model(img)
    logits = tf.math.divide(logits, temp_value[3])
    pred = model3.apply_softmax(logits)
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 4:
    logits = model4.model(img)
    logits = tf.math.divide(logits, temp_value[4])
    pred = model4.apply_softmax(logits)
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 5:
    logits = model5.model(img)
    logits = tf.math.divide(logits, temp_value[5])
    pred = model5.apply_softmax(logits)
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 6:
    logits = model6.model(img)
    logits = tf.math.divide(logits, temp_value[6])
    pred = model6.apply_softmax(logits)
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 7:
    logits = model7.model(img)
    logits = tf.math.divide(logits, temp_value[7])
    pred = model7.apply_softmax(logits)
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 8:
    logits = model8.model(img)
    logits = tf.math.divide(logits, temp_value[8])
    pred = model8.apply_softmax(logits)
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

threshold_t = treshold_value/len(classifier_avg_t)

print('Threshold(with temp scaling):', threshold_t)

10817it [00:18, 577.82it/s]

[910.7833430767059, 792.3951269239187, 910.187284886837, 951.6759190261364, 820.0793315172195, 847.958394497633, 799.5131324231625, 930.3852233886719, 835.0283146351576]
[1230, 1324, 1211, 1229, 1139, 1084, 1157, 1265, 1178]
Each Classifier Average:  [0.7404742626639885, 0.5984857454108148, 0.7515997397909472, 0.7743498120635772, 0.7199994130967687, 0.7822494414184806, 0.6910225863640125, 0.7354823900305707, 0.708852559113037]
Threshold(with temp scaling): 0.7225017722169108


In [ ]:
pickle_out = open("reference_vector_ood.pickle","wb")
pickle.dump([classifier_avg, classifier_avg_t], pickle_out)
pickle_out.close()